# Worksheet 08

Name:  
UID: 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [43]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.921945647154406, 5.734093711864335, 5.179595749456672, 5.41548951127398, 8.048039917313872, 4.940694023249918, 5.519081867382752, 3.3820196640373306, 3.7857013174678062, 4.680525119943022]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [44]:
s2 = np.random.normal( 8, 1, 10 ).tolist()
print(s2)

[9.615496228832093, 7.936116279898326, 7.861616573328665, 8.924757180660473, 9.017070923408541, 8.18019234705216, 8.021606351236032, 6.797770447668544, 6.854210950743029, 8.5927870360461]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [45]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[8.5927870360461, 4.680525119943022, 3.7857013174678062, 6.854210950743029, 3.3820196640373306, 6.797770447668544, 8.021606351236032, 8.18019234705216, 9.017070923408541, 5.519081867382752]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

mean_1 = 5; variance_1 = 1; P(S_1) = 0.5 

mean_2 = 8; variance_2 = 1; P(S_2) = 0.5

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [46]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 =  [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[8.5927870360461, 6.854210950743029, 6.797770447668544, 8.021606351236032, 8.18019234705216, 9.017070923408541]
[4.680525119943022, 3.7857013174678062, 3.3820196640373306, 5.519081867382752]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 7.910606342692401,  mean_2 = 4.3418319922077275
var_1 = 0.6881678226321067,  var_2 = 0.6827878340173213


/Users/xavierohan/anaconda3/envs/mlproject/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


It is close to the true values

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [47]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( (pdf_i[0] * prob_s[0]) / prob_x)
    prob_s1_x.append( (pdf_i[1] * prob_s[1]) / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  8.5927870360461
probability of observing that point if it came from cluster 0 =  0.3546748814827167
probability of observing that point if it came from cluster 1 =  2.2369385615856214e-09
point =  4.680525119943022
probability of observing that point if it came from cluster 0 =  9.532379143238585e-06
probability of observing that point if it came from cluster 1 =  0.5166459350712531
point =  3.7857013174678062
probability of observing that point if it came from cluster 0 =  9.150383892386778e-09
probability of observing that point if it came from cluster 1 =  0.4193402932157218
point =  6.854210950743029
probability of observing that point if it came from cluster 0 =  0.17844811623617607
probability of observing that point if it came from cluster 1 =  0.0006707883163890778
point =  3.3820196640373306
probability of observing that point if it came from cluster 0 =  2.2892050478143019e-10
probability of observing that point if it came from cluster 1 =  0.21753410439307624
point 

Even though the probability values are not matching with the ground truth, the maximum probability is given to the same cluster for both

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [48]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum(prob_s0_x[i] * (data[i] - mean[0])**2 for i in range(len(data))) / sum(prob_s0_x) , sum(prob_s1_x[i] * (data[i] - mean[1])**2 for i in range(len(data))) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 7.905592364070848,  mean_2 = 4.341207531343438
var_1 = 0.7005948807247331,  var_2 = 0.688740061039152


The values are not too far away from the values obtained from Kmeans


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [49]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( (pdf_i[0] * prob_s[0]) / prob_x)
    prob_s1_x.append( (pdf_i[1] * prob_s[1]) / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  8.5927870360461
probability of observing that point if it came from cluster 0 =  0.35198416985847936
probability of observing that point if it came from cluster 1 =  3.078276003419022e-09
point =  4.680525119943022
probability of observing that point if it came from cluster 0 =  1.4254723608104276e-05
probability of observing that point if it came from cluster 1 =  0.5130375684771855
point =  3.7857013174678062
probability of observing that point if it came from cluster 0 =  1.7630898915505395e-08
probability of observing that point if it came from cluster 1 =  0.41840222342888617
point =  6.854210950743029
probability of observing that point if it came from cluster 0 =  0.18467449935170488
probability of observing that point if it came from cluster 1 =  0.0007446918152725693
point =  3.3820196640373306
probability of observing that point if it came from cluster 0 =  5.04231754501455e-10
probability of observing that point if it came from cluster 1 =  0.21962965290123776
point

The probabilities have a higher value at the clusters that had higher probability.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [50]:
hard_assignments = [0 if prob_s0_x[i] > prob_s1_x[i] else 1 for i in range(len(data))]

# Print the hard assignments
for i in range(len(data)):
    print("Data Point:", data[i])
    print("Assigned Cluster:", hard_assignments[i])
    print()

Data Point: 8.5927870360461
Assigned Cluster: 0

Data Point: 4.680525119943022
Assigned Cluster: 1

Data Point: 3.7857013174678062
Assigned Cluster: 1

Data Point: 6.854210950743029
Assigned Cluster: 0

Data Point: 3.3820196640373306
Assigned Cluster: 1

Data Point: 6.797770447668544
Assigned Cluster: 0

Data Point: 8.021606351236032
Assigned Cluster: 0

Data Point: 8.18019234705216
Assigned Cluster: 0

Data Point: 9.017070923408541
Assigned Cluster: 0

Data Point: 5.519081867382752
Assigned Cluster: 1

